In [ ]:
%pip install git+https://github.com/openai/CLIP.git

In [1]:
import torch
import clip
from PIL import Image, ImageDraw

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

In [4]:
image = Image.open("D:/Work/coding/vuzz/new/MIREA_7_semester_VT/rsppr/SAFE-MACS/data/cppe5/test/images/540_jpg.rf.c38ef2dc044f14b40238f33a205656db.jpg")
preprocess_image = preprocess(image).unsqueeze(0).to(device)
text = clip.tokenize(["a coverall", "a face shield", "gloves", "goggles", "a mask"]).to(device)

In [ ]:
with torch.no_grad():
    image_features = model.encode_image(preprocess_image)
    text_features = model.encode_text(text)
    
    logits_per_image, logits_per_text = model(preprocess_image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

print("Label probs:", probs)

Label probs: [[0.2578  0.5293  0.02289 0.0683  0.1218 ]] tensor([[26.2656, 26.9844, 23.8438, 24.9375, 25.5156]], device='cuda:0',
       dtype=torch.float16)


In [ ]:
draw = ImageDraw.Draw(image)

# for result in results:
#     boxes = result['boxes']
#     labels = result['labels']
#     scores = result['scores']

for box, label, score in zip(boxes, labels, scores):
    box = box.cpu().numpy().astype(int)
    
    draw.rectangle([(box[0], box[1]), (box[2], box[3])], outline="red", width=2)
    
    text = f"{label}: {score.item():.2f}"
    draw.text((box[0], box[1] - 10), text, fill="red")

# image.save("output_image.jpg")
# image.show()
display(image)

TRAIN

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import clip
from transformers import CLIPProcessor, CLIPModel

class CustomDataset(Dataset):
    def __init__(self, image_paths, captions, transform=None):
        self.image_paths = image_paths
        self.captions = captions
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, self.captions[idx]

# Load pre-trained CLIP model
model, preprocess = clip.load("ViT-B/32", device="cuda" if torch.cuda.is_available() else "cpu")


# Path
image_paths = ...
captions = ...

# Create custom dataset and dataloader
dataset = CustomDataset(image_paths, captions, transform=preprocess)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Training loop
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
num_epochs = 10

for epoch in range(num_epochs):
    for images, texts in dataloader:
        optimizer.zero_grad()
        
        # Forward pass
        image_features = model.encode_image(images)
        text_features = model.encode_text(texts)
        
        # Compute loss (implement contrastive loss here)
        loss = ...
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

# Save the fine-tuned model
torch.save(model.state_dict(), 'fine_tuned_clip.pth')